In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime 

In [ ]:

from qiskit import Aer
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, QAOA, SamplingVQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import algorithm_globals
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance
from qiskit.result import QuasiDistribution
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.applications import OptimizationApplication
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_aer.primitives import Sampler

In [ ]:
# establishing the number of assets (same as # of cubits)

num_assets = 6
seed = 123

# select randomly generated data through RandomDataProvider

stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = RandomDataProvider(
    tickers=stocks,
    start=datetime.datetime(2021, 1, 1),
    end=datetime.datetime(2021, 1, 30),
    seed=seed,
)

In [ ]:
# calculate expected return and covariance from generated data

data.run()
exp_return = data.get_period_return_mean_vector()
cov = data.get_period_return_covariance_matrix()
x = np.array(data._data)

In [ ]:

sns.lineplot(x.T)

In [ ]:
exp_return

In [ ]:
# plot sigma
plt.imshow(cov)
plt.colorbar()
plt.show()

In [ ]:
# establishing parameters for optimizer
rf = 0.5  
budget = num_assets // 2    
portfolio = PortfolioOptimization(
    expected_returns=exp_return, covariances=cov, risk_factor=rf, budget=budget
)
qp = portfolio.to_quadratic_program()
qp

In [ ]:
qp.variables_index



In [ ]:
# organizing print results
def print_result(result):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )
    print("\n----------------- Full result ---------------------")
    print("selection\tvalue\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = portfolio.to_quadratic_program().objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))

In [ ]:
algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
qaoa_mes = QAOA(sampler=Sampler(), optimizer=cobyla, reps=3)
qaoa = MinimumEigenOptimizer(qaoa_mes)
result = qaoa.solve(qp)

print_result(result)
